In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [3]:
comment_classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [4]:
train =  pd.read_csv('./train.csv').fillna(' ')

In [5]:
test = pd.read_csv('./test.csv').fillna(' ')

In [6]:
train_comments = train['comment_text']
test_comments = test['comment_text']

In [7]:
all_comments = pd.concat([train_comments,test_comments])

In [8]:
type(all_comments)

pandas.core.series.Series

# Convert a collection of raw documents to a matrix of TF-IDF features.

In [9]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=30000)

In [10]:
word_vectorizer?

In [11]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)

In [14]:
vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=4)

In [15]:
vectorizer.fit(all_comments)

FeatureUnion(n_jobs=4,
       transformer_list=[('tfidfvectorizer-1', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=30000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=...
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None))],
       transformer_weights=None)

In [20]:
train_features = vectorizer.transform(train_comments)
test_features = vectorizer.transform(test_comments)


In [16]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [21]:
for class_name in comment_classes:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(
        classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('submission_logistic.csv', index=False)

CV score for class toxic is 0.9788002022449215
CV score for class severe_toxic is 0.9887814831547654
CV score for class obscene is 0.9903320988155363
CV score for class threat is 0.9887827992117818
CV score for class insult is 0.9828864077305525
CV score for class identity_hate is 0.9831331367400665
Total CV score is 0.9854526879829373


In [22]:
classifierfier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
from keras.models import model_from_json

/home/erh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
model_json = classifier.to_json()
with open("model_GRU.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_GRU.h5")
print("Saved model to disk")